[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/corazzon/finance-data-analysis/blob/main/3.3%20%EB%84%A4%EC%9D%B4%EB%B2%84%EA%B8%88%EC%9C%B5%20%EA%B0%9C%EB%B3%84%EC%A2%85%EB%AA%A9%20%EC%88%98%EC%A7%91-input.ipynb)


## 네이버 금융 개별종목 수집
* FinanceDataReader를 통해 수집했던 데이터를 네이버 증권 웹 페이지를 통해 직접 수집합니다.


### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 2020년 주요 상장종목
    * 빅히트 : https://finance.naver.com/item/main.nhn?code=352820
    * 카카오게임즈 : https://finance.naver.com/item/main.nhn?code=293490
    * SK바이오팜 : https://finance.naver.com/item/main.nhn?code=326030

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd

## 수집할 URL 정하기

In [19]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "352820"
# item_name = "빅히트"

item_code = "326030"
item_name = "SK바이오팜"

# 종목 URL 만들기
url = "https://finance.naver.com/item/sise_day.naver?code=005930&page=5"


## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [22]:
import requests

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

response = requests.get(url, headers=headers) # <Response [200]> : 정상적으로 잘 받았다라는 의미
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 증권</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230718202221/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230718202221/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230718202221/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230718202221/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230718202221/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230718202221/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20230718202221/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

* [Beautiful Soup Documentation — Beautiful Soup 4.9.0 documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)

## pandas 코드 한 줄로 데이터 수집하기

In [7]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.


# url = "https://finance.naver.com/item/sise.naver?code=005930"
table = pd.read_html(url, encoding='CP949') 
len(table) # 일별 시세를 보기 위해 총 12개의 Table(표)를 불러왔다 

12

In [17]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[0] # [0]~[11]: 일별시세는 찾을 수 없었다

,업체명,거래량,전일비
0,에코프로,1146000.0,"하향 82,000"
1,에코프로비엠,433000.0,"하향 22,000"
2,포스코DX,34450.0,"하향 1,050"
3,씨젠,24200.0,"상향 4,540"
4,엘앤에프,259000.0,"하향 4,000"
5,NaN,NaN,NaN
6,유안타제14호..,4810.0,"상향 2,810"
7,에이엘티,25050.0,상향 50
8,ISC,107100.0,"상향 15,200"
9,포스코엠텍,37850.0,"하향 1,300"


In [ ]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
# temp

## 페이지별 데이터 수집 함수 만들기

In [ ]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

# def get_day_list(item_code, page_no):
# """
# 일자별 시세를 페이지별로 수집
# """ 

In [ ]:
# 함수가 잘 만들어졌는지 확인하기


## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [ ]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []


## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [ ]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.


<img src="https://pandas.pydata.org/docs/_images/08_concat_row.svg">

In [ ]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [ ]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
# df["종목코드"]
# df["종목명"]

## 컬럼 순서 변경하기

In [ ]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.


In [ ]:
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [ ]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인


## 기술통계값 구하기

In [ ]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [ ]:
# 거래량의 과학적 기수법 읽기


## 최근 날짜 구해서 파일명 만들기

In [ ]:
# 날짜 column의 첫 row값 확인
# date

In [ ]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
# file_name


## 파일로 저장하기

In [ ]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.


In [ ]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.


## 전체 과정을 하나의 함수로 만들기

In [ ]:
# def get_item_list(item_code, item_name):
#     """
#     일별 시세를 수집하는 함수
#     """

In [ ]:
# item_code = "352820"
# item_name = "하이브"
# 빅히트의 사명이 하이브로 변경되었습니다.

item_code = "326030"
item_name = "SK바이오팜"


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [ ]:
# 도움말 보기
# get_item_list?


In [ ]:
# 소스코드 보기
# get_item_list??
